In [1]:
import logging
import pickle
import time
from statistics import mean, median

import en_core_web_sm
import gensim.utils
import gensim.models.keyedvectors
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import re
import seaborn as sns
import spacy
import string

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from imblearn.under_sampling import RandomUnderSampler
from keras_preprocessing.sequence import pad_sequences

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk.tokenize import word_tokenize, sent_tokenize

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score, precision_score, recall_score
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
nltk.download('punkt')

logging.basicConfig(format="%(message)s")
logger = logging.getLogger()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Kika\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
nlp = spacy.load("en_core_web_sm")

In [3]:
def wordnet_sim(syn1, syn2, lemma, sim_type='path'):
    if sim_type == "path":
        sim = syn1.path_similarity(syn2)
    elif sim_type == "wup":
        sim = syn1.wup_similarity(syn2)
    elif sim_type == "lch":
        sim = syn1.lch_similarity(syn2)     
    else:
        raise Exception("'{}' is not a valid similarity type.".format(sim_type))
    return sim

In [4]:
def search(word):
    # POS tagging
    global tok

    synonyms = [word]
    doc = nlp(word)
    
    for tok in doc:
        synsets_ = []
        
        if tok.pos_ in ("NOUN", "VERB", "ADJ", "ADV", "PROPN"):
            
            try:
                synsets_ = wordnet.synsets(word)
            
                if len(synsets_) == 0:
                    # try with Morphy. e.g.: 'denied' -> 'deny'
                    word_morphed = wordnet.morphy(word)
                    if word_morphed is None:
                        return synonyms
                    synsets_ = wordnet.synsets(word_morphed)
            except Exception as e:
                logger.error("Error retrieving synsets for {} as a '{}': {}".format(word, tok.pos_, e))
                logger.exception(e)

        try:
            for syn in synsets_:
                for lemma in syn.lemmas():
                    sim = wordnet_sim(syn, synsets_[0], lemma)
                    if sim and sim > 0.5:
                        if (lemma.name()).lower() not in synonyms:
                            synonyms.append((lemma.name()).lower())

        except Exception as e:
            logger.error("Error while looking for synonyms: {}".format(e))
            logger.exception(e)

    return synonyms

In [5]:
def get_synonyms(word):
    synonyms = search(word)
    
    word = synonyms[0]
    if len(synonyms) > 1 and (word not in synonyms_all):
        synonyms_all[word] = synonyms[1:6]
    
    return None

## 3 - Načítanie upravených viet a vytvorenie zoznamu synoným zo synsetov (potrebujem vety A) 2. krok)

In [6]:
#Načítanie viet
import pickle
with open("files/sentences", "rb") as fp:   # Unpickling
    sentences = pickle.load(fp)

In [7]:
len(sentences)

10000

In [8]:
synonyms_all = dict()
for i in range(len(sentences)):
    for j in range(len(sentences[i])):
        get_synonyms(sentences[i][j])
    if i % 1000 == 0:
        print(i, "of", len(sentences))
        
        
with open("files/synonyms", "wb") as fp:
    pickle.dump(synonyms_all, fp)

0 of 10000
1000 of 10000
2000 of 10000
3000 of 10000
4000 of 10000
5000 of 10000
6000 of 10000
7000 of 10000
8000 of 10000
9000 of 10000


In [9]:
print("The number of words for which synonyms were found: ",len(synonyms_all))

The number of words for which synonyms were found:  9704
